In [ ]:
import os
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\Figure\inversion\GAT')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import argparse
from collections import defaultdict

In [ ]:
def read_bed(path):
    """
    Reads a BED file (tab-delimited) and returns a dictionary mapping each chromosome
    to a list of (start, end) tuples.
    """
    d = defaultdict(list)
    with open(path, 'r') as f:
        for line in f:
            if line.strip() == "" or line.startswith("#"):
                continue
            parts = line.strip().split()
            chrom = parts[0]
            start = int(parts[1])
            end = int(parts[2])
            d[chrom].append((start, end))
    return d

In [ ]:
def total_overlap(segments, annotations):
    """
    Given two dicts {chrom: [(start, end), ...]}, compute the total number of overlapping base pairs.
    """
    total = 0
    for chrom, seg_list in segments.items():
        if chrom not in annotations:
            continue
        annot_list = annotations[chrom]
        for seg_start, seg_end in seg_list:
            for ann_start, ann_end in annot_list:
                if ann_end <= seg_start or ann_start >= seg_end:
                    continue
                overlap_start = max(seg_start, ann_start)
                overlap_end = min(seg_end, ann_end)
                total += (overlap_end - overlap_start)
    return total

In [ ]:
def sample_random_segments(workspace, lengths):
    """
    Randomly sample segments from the workspace, matching each length in the 'lengths' list.
    Each sampled segment is chosen so that it fits entirely within a single workspace interval.
    Returns a dict in the same format as 'read_bed'.
    """
    sampled = defaultdict(list)
    # Flatten workspace intervals for random choice
    intervals = []
    for chrom, segs in workspace.items():
        for start, end in segs:
            intervals.append((chrom, start, end))

    for length in lengths:
        while True:
            chrom, w_start, w_end = random.choice(intervals)
            if w_end - w_start < length:
                continue
            rand_start = random.randint(w_start, w_end - length)
            rand_end = rand_start + length
            sampled[chrom].append((rand_start, rand_end))
            break
    return sampled

In [ ]:
def compute_permutation_stats(inv_dict, anno_dict, ws_dict, num_permutations=1000):
    """
    Computes observed overlap and expected overlap distribution by permutation.
    Returns a dict with statistics and the list of permutation overlaps.
    """
    # Observed overlap
    observed = total_overlap(inv_dict, anno_dict)
    # Collect inversion lengths
    lengths = []
    for seg_list in inv_dict.values():
        for (start, end) in seg_list:
            lengths.append(end - start)
    # Permutation: sample random segments and compute overlap each time
    expected_list = []
    for _ in range(num_permutations):
        rnd_segments = sample_random_segments(ws_dict, lengths)
        expected_list.append(total_overlap(rnd_segments, anno_dict))
    expected_mean = np.mean(expected_list)
    expected_std = np.std(expected_list, ddof=1)
    fold_change = observed / expected_mean if expected_mean > 0 else float('inf')
    p_value = sum(1 for x in expected_list if x >= observed) / num_permutations
    return {
        "observed": observed,
        "expected_list": expected_list,
        "expected_mean": expected_mean,
        "expected_std": expected_std,
        "fold_change": fold_change,
        "p_value": p_value
    }

In [ ]:
def occupancy_profile(inv_dict, anno_dict, flank=1_000_000, step=1000):
    """
    Computes average occupancy of annotations around inversion breakpoints.
    Returns two lists: x_coords (in kb) and occupancy values.
    """
    num_bins = (2 * flank) // step
    sum_overlap = np.zeros(num_bins, dtype=float)
    num_inv = 0

    for chrom, seg_list in inv_dict.items():
        if chrom not in anno_dict:
            continue
        ann_list = anno_dict[chrom]
        for start, end in seg_list:
            num_inv += 1
            center = (start + end) // 2
            for i in range(num_bins):
                rel_start = center - flank + i * step
                rel_end = rel_start + step
                if rel_end < 0:
                    continue
                overlap_bp = 0
                for ann_start, ann_end in ann_list:
                    if ann_end <= rel_start or ann_start >= rel_end:
                        continue
                    overlap_bp += min(ann_end, rel_end) - max(ann_start, rel_start)
                sum_overlap[i] += overlap_bp

    avg_occ = sum_overlap / (step * num_inv)
    x_kb = [(-flank // 1000) + i for i in range(num_bins)]
    return x_kb, avg_occ

In [ ]:
def plot_permutation_histogram(expected_list, observed, expected_mean, outpath1,outpath2):
    """
    Plots a histogram of the permutation results and marks observed and expected mean.
    """
    plt.rcParams['pdf.fonttype'] = 42  # TrueType
    plt.rcParams['ps.fonttype'] = 42
    plt.rcParams['font.family'] = 'Arial'
    plt.figure(figsize=(5, 3))
    plt.hist(expected_list, bins=30, color='lightgray', edgecolor='black')
    plt.axvline(observed, color='red', linestyle='--', linewidth=2, label=f"Observed = {observed}")
    plt.axvline(expected_mean, color='blue', linestyle='-', linewidth=2, label=f"Expected = {expected_mean:.1f}")
    plt.xlabel("Total overlap (bp)")
    plt.ylabel("Frequency")
    # plt.title("Permutation Test: Repeat Enrichment at Inversion Breakpoints")
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath1)
    plt.savefig(outpath2)
    plt.close()

In [ ]:
inv_dict = read_bed("inv.tmp.bed")
anno_dict = read_bed("HJD.sd.regions.bed")
# anno_dict = read_bed("HJD.level3.Copia.txt")
# anno_dict = read_bed("HJD.level4.Gypsy.txt")
anno_dict = read_bed("HJD.level17.DNA.txt")

ws_dict = read_bed("workspace1.bed")
num_perm=1000
out_prefix="40kDNA"

# Compute permutation statistics
stats = compute_permutation_stats(inv_dict, anno_dict, ws_dict, num_permutations=num_perm)
observed = stats["observed"]
expected_list = stats["expected_list"]
expected_mean = stats["expected_mean"]
expected_std = stats["expected_std"]
fold_change = stats["fold_change"]
p_value = stats["p_value"]
print(f"observed:{observed}")
print(f"expected_mean:{expected_mean}")
print(f"expected_std:{expected_std}")
print(f"fold_change:{fold_change}")
print(f"p_value:{p_value}")
# Plot histogram (similar to Figure 4A)
hist_out1 = f"{out_prefix}_figureA_histogram.png"
hist_out2 = f"{out_prefix}_figureA_histogram.pdf"
plot_permutation_histogram(expected_list, observed, expected_mean, hist_out1, hist_out2)